In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder
import librosa

# Load the saved model
model = load_model('/kaggle/input/h5model/trymodel.h5')

# Function to extract MFCC features
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

# Assuming you have a test dataset in the same directory format as the training dataset
test_paths = []
test_labels = []
for dirname, _, filenames in os.walk('/kaggle/input/toronto-emotional-speech-set-tess'):
    for filename in filenames:
        test_paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        label = label.split('.')[0]
        test_labels.append(label.lower())

# Create a dataframe for the test dataset
test_df = pd.DataFrame()
test_df['speech'] = test_paths
test_df['label'] = test_labels

# Extract MFCC features for the test dataset
X_test_mfcc = test_df['speech'].apply(lambda x: extract_mfcc(x))

X_test = [x for x in X_test_mfcc]
X_test = np.array(X_test)
X_test = np.expand_dims(X_test, -1)

# One-hot encode the labels for the test dataset
enc = OneHotEncoder()
y_test = enc.fit_transform(test_df[['label']])
y_test = y_test.toarray()

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Test Accuracy: {accuracy * 100:.2f}%')


2024-07-26 09:32:10.396980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 09:32:10.397103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 09:32:10.560862: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9995 - loss: 0.0013
Test Accuracy: 99.86%


In [2]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 256)            │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 305,801 (1.17 MB)

 Trainable params: 305,799 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
import os
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import Accuracy
from sklearn.preprocessing import OneHotEncoder
import librosa

# Load the saved model
model = load_model('/kaggle/input/obfuscate/obfuscated_model.h5')

# Compile the model
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=[Accuracy()])

# Function to extract MFCC features
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

# Assuming you have a test dataset in the same directory format as the training dataset
test_paths = []
test_labels = []
for dirname, _, filenames in os.walk('/kaggle/input/toronto-emotional-speech-set-tess'):
    for filename in filenames:
        test_paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        label = label.split('.')[0]
        test_labels.append(label.lower())

# Create a dataframe for the test dataset
test_df = pd.DataFrame()
test_df['speech'] = test_paths
test_df['label'] = test_labels

# Extract MFCC features for the test dataset
X_test_mfcc = test_df['speech'].apply(lambda x: extract_mfcc(x))

X_test = [x for x in X_test_mfcc]
X_test = np.array(X_test)
X_test = np.expand_dims(X_test, -1)

# One-hot encode the labels for the test dataset
enc = OneHotEncoder()
y_test = enc.fit_transform(test_df[['label']])
y_test = y_test.toarray()

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Test Accuracy: {accuracy * 100:.2f}%')


175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0000e+00 - loss: 2.0763
Test Accuracy: 0.00%


In [6]:
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 40, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ YsczLmDnnN (LSTM)   │ (None, 256)       │    264,192 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ CrlpvTsvb5 (Dense)  │ (None, 42)        │     10,794 │ YsczLmDnnN[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tqL4mzCkk7 (Add)    │ (None, 42)        │          0 │ CrlpvTsvb5[0][0], │
│                     │                   │            │ CrlpvTsvb5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GYtLpC6LFI          │ (None, 42)        │          0 │ tqL4mzCkk7[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 2nlEe0iGMj (Dense)  │ (None, 91)        │      3,913 │ GYtLpC6LFI[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hbkh4Dm8b0 (Add)    │ (None, 91)        │          0 │ 2nlEe0iGMj[0][0], │
│                     │                   │            │ 2nlEe0iGMj[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ c1SN7tm2XS (Dense)  │ (None, 128)       │     11,776 │ hbkh4Dm8b0[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 2tEHsY7TpN (Add)    │ (None, 128)       │          0 │ c1SN7tm2XS[0][0], │
│                     │                   │            │ c1SN7tm2XS[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gvFJej5Aps          │ (None, 128)       │          0 │ 2tEHsY7TpN[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RscI0Cola3 (Dense)  │ (None, 82)        │     10,578 │ gvFJej5Aps[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ XngGkIgJZc (Dense)  │ (None, 64)        │      5,312 │ RscI0Cola3[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ BbrkeqXc1w          │ (None, 64)        │          0 │ XngGkIgJZc[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NzC2G9fEc2 (Dense)  │ (None, 7)         │        455 │ BbrkeqXc1w[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 307,020 (1.17 MB)

 Trainable params: 307,020 (1.17 MB)

 Non-trainable params: 0 (0.00 B)